In [3]:
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import gzip
from collections import defaultdict
import random

from sklearn.feature_extraction.text import TfidfVectorizer
import urllib
import scipy.optimize
from collections import defaultdict # Dictionaries with default values
import nltk
import string
from nltk.stem.porter import *
from sklearn import linear_model
import ast

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report, precision_recall_fscore_support

from sklearn.linear_model import LogisticRegression

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\fengj\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
rent = pd.read_csv('clean.csv')
rent.review_summary = rent.review_summary.fillna('')
rent.review_text = rent.review_text.fillna('')

In [5]:
rent = rent.sample(frac=1).reset_index(drop=True)
train = rent[:180000]
valid = rent[180000:]

In [6]:
y_train = train.fit
y_valid = valid.fit

In [14]:
train

,fit,user_id,item_id,weight,rating,rented for,review_text,body type,review_summary,category,height,size,age,review_date,size_guess_fit,bust_num,bust_char
0,fit,208774,153475,15.8725,10.0,wedding,The 10l was snug on my chest 36d but the 12l w...,hourglass,Felt like a movie star! Dress came as ordered ...,culott,9.37,12.754825,3.526361,"June 29, 2014",37.997325,38,d
1,fit,847724,1436642,1.3605,10.0,wedding,"I want to own this dress, I loved it so much! ...",athletic,"Great pink color, worked wonderfully for a bab...",culott,1.75,-4.245175,3.555348,"April 8, 2017",-1.134675,36,b
2,fit,2635,1996590,-0.9070,8.0,party,I got a lot of compliments about how cute this...,full bust,Comfortable but fun!,culott,-13.49,-8.245175,3.663562,"January 28, 2013",-22.642175,36,c
3,large,999231,1852099,-0.9070,10.0,everyday,Super comfortable dress by Nicole Miller. Runn...,other,STYLIST REVIEW!!,culott,-0.79,-4.245175,3.295837,"September 9, 2016",-5.942175,34,dd
4,fit,105485,178058,0.0000,8.0,date,"This dress fit great, except it was a little t...",pear,"Comfy, but tight in bust.",culott,-5.87,3.754825,3.713572,"September 16, 2013",-2.115175,34,d
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179995,fit,676596,136110,14.9655,10.0,party,"Perfect flare dress, sleeves the perfect tight...",full bust,Beautiful dress. The color was amazing and the...,culott,1.75,26.754825,3.295837,"March 15, 2016",43.470325,38,d
179996,large,748397,2696735,-5.4420,8.0,other,Have this currently in my Unlimited Order. 4 i...,pear,Great to throw on top of everything,coat,4.29,-4.245175,3.610918,"February 2, 2015",-5.397175,34,c
179997,large,135436,955151,-5.8955,10.0,everyday,This dress runs slightly big. I loved the loo...,athletic,Cute summer date night dress,culott,-0.79,-4.245175,3.737670,"July 23, 2017",-10.930675,34,d
179998,fit,946761,1586334,0.0000,8.0,party,"I did get a lot of compliments on the dress, b...",other,The dress was a great pick for an evening gala...,culott,4.29,7.754825,3.526361,"November 5, 2017",12.044825,36,c


In [7]:
tfidf = TfidfVectorizer(stop_words = 'english', sublinear_tf = True, max_features = 1000, tokenizer = word_tokenize) 
Train_X_summary = tfidf.fit_transform(train.review_summary).toarray()

In [8]:
tfidf2 = TfidfVectorizer(stop_words = 'english', sublinear_tf = True, max_features = 1000, tokenizer = word_tokenize) 
Train_X_text = tfidf2.fit_transform(train.review_text).toarray()

In [9]:
category_train = np.array([train['category'].to_list()]).transpose()
weight_train = np.array([train['weight'].to_list()]).transpose()
age_train = np.array([train['age'].to_list()]).transpose()
height_train = np.array([train['height'].to_list()]).transpose()

In [10]:
combined_text = np.hstack((Train_X_summary, Train_X_text))

In [11]:
other_feature = np.hstack(( category_train,weight_train,age_train,height_train))

In [15]:
combined_text

(180000, 2000)

In [16]:
other_feature.shape

(180000, 4)

In [29]:
lstsss = []
for i in range(180000):
    lst = list(combined_text[i]) + list(other_feature[i])
    lstsss.append(lst)

In [31]:
len(lstsss)

180000

In [36]:
final_feature = lstsss[:120000]

In [37]:
clf = LogisticRegression()
clf.fit(final_feature, y_train)

MemoryError: Unable to allocate 1.79 GiB for an array with shape (120000, 2004) and data type float64

In [73]:
Valid_X_summary = tfidf.transform(valid.review_summary).toarray()

In [74]:
Valid_X_text = tfidf2.transform(valid.review_text).toarray()

In [76]:
combined_text_valid = np.hstack((Valid_X_summary, Valid_X_text))

In [77]:
category_valid = np.array([valid['category'].to_list()]).transpose()
weight_valid = np.array([valid['weight'].to_list()]).transpose()
age_valid = np.array([valid['age'].to_list()]).transpose()
height_valid = np.array([valid['height'].to_list()]).transpose()

In [80]:
other_feature_valid = np.hstack((category_valid,weight_valid,age_valid,height_valid))

In [81]:
final_feature_valid = np.hstack((combined_text_valid, other_feature_valid))

In [83]:
clf.score(final_feature_valid, y_valid)

C:\Users\Kennnnnnn\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:268: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using your_array = your_array.astype(np.float64).
  X = check_array(X, accept_sparse='csr')


ValueError: X has 2004 features per sample; expecting 2000

In [20]:
y_valid.value_counts()

fit      9235
small    1698
large    1611
Name: fit, dtype: int64

In [21]:
9235 / (len(y_valid))

0.7362085459183674

In [22]:
clf = LogisticRegression(max_iter = 10000, C=0.1)
clf.fit(combined_text, y_train)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [23]:
clf.score(combined_text_valid, y_valid)

0.8038903061224489